Caso Advanced Analytics: 
Predicción de Fugas de Clientes para una Institución Financiera
======

Autor: Rolf Traeger Brautigam
=========

# Inicio
## Import librarys

In [1]:
# ! conda install plotnine
# ! conda install pandas
# ! conda install scikit-learn
# ! conda install -c anaconda git


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from plotnine import (
ggplot,
geom_histogram,
aes,
geom_col,
theme,
element_text,
ggtitle,
scale_x_discrete,
facet_grid,
position_dodge,
)

## Funciones utiles

In [4]:
import exploratory_lib as exp_lib

In [5]:
dic_med=exp_lib.class_diccionarios()
dic_med.set_list_num(list(range(1,13)))
dic_med.set_list_name(exp_lib.array_mes_name)
dic_med.set_dictionary()

In [6]:
dic_med.int_to_name(1)

'Enero'

In [7]:
dic_dianom=exp_lib.class_diccionarios()
dic_dianom.set_list_num(list(range(1,8)))
dic_dianom.set_list_name(['Lunes', 'Martes', 'Miercoles','Jueves','Viernes', 'Sabado', 'Domingo'])
dic_dianom.set_dictionary()
dic_dianom.name_to_num("LuneS")

1

## Extract CSV y transform using owns librarys

In [8]:
list_name_file_1 = [
    'aportes_rescates_12m',      #.csv
    'cuenta_banco',      #.csv
    'demograficos',      #.csv
    'deudas',        #.csv
    'id_label',      #.csv
    ]
list_name_file_cierre_patrimonio=[
'cierre_patrimonio_202202',        #.csv
'cierre_patrimonio_202203',        #.csv
'cierre_patrimonio_202204',        #.csv
'cierre_patrimonio_202205',        #.csv
'cierre_patrimonio_202206',        #.csv
'cierre_patrimonio_202207',        #.csv
'cierre_patrimonio_202208',        #.csv
'cierre_patrimonio_202209',        #.csv
'cierre_patrimonio_202210',        #.csv
'cierre_patrimonio_202211',        #.csv
'cierre_patrimonio_202212',        #.csv
'cierre_patrimonio_202301',        #.csv
'cierre_patrimonio_202302',        #.csv
]

In [9]:
aportes_rescates_12m = pd.read_csv("bases/aportes_rescates_12m.csv",
                        encoding="utf-8",sep=",",nrows=0, 
                        index_col=0)
# exp_lib.extract_csv("bases","aportes_rescates_12m",',')
cuenta_banco = pd.read_csv("bases/cuenta_banco.csv",
                        encoding="utf-8",sep=",",nrows=0
                        ,index_col=None)
# exp_lib.extract_csv("bases","cuenta_banco",',')
demograficos =  pd.read_csv("bases/demograficos.csv",
                        encoding="utf-8",sep=",",nrows=0
                        ,index_col=None)
deudas =   pd.read_csv("bases/demograficos.csv",
                        encoding="utf-8",sep=",",nrows=0
                        ,index_col=None)
# exp_lib.extract_csv("bases","deudas",',')
id_label = pd.read_csv("bases/id_label.csv",
                        encoding="utf-8",sep=",",nrows=0
                        ,index_col=None)


In [10]:
aportes_rescates_12m = exp_lib.extract_csv("bases","aportes_rescates_12m",',',0)
cuenta_banco = exp_lib.extract_csv("bases","cuenta_banco",',')
demograficos = exp_lib.extract_csv("bases","demograficos",',')
deudas = exp_lib.extract_csv("bases","deudas",',')
id_label = exp_lib.extract_csv("bases","id_label",',')


Dimensiones: (37146, 3)
Columnas 
 n_rescates, n_aportes, id
Dimensiones: (71664, 2)
Columnas 
 id, clasificacion_cliente
Dimensiones: (36970, 10)
Columnas 
 id, fec_nacim, flg_empleado, gls_nacioncorta, gls_residencialarga, gls_profesionlarga, cod_sexo, gls_cod_estcivillarga, gls_niv_educaclarga, gls_sep_bieneslarga
Dimensiones: (367543, 16)
Columnas 
 id, periodo, entidades, disponible, deuda_1, deuda_2, deuda_3, deuda_4, deuda_5, mora_1, mora_2, mora_3, mora_4, mora_5, mora_6, mora_7
Dimensiones: (37146, 2)
Columnas 
 id, label


In [11]:
df_list_cierre_patrimonio = [
    exp_lib.extract_csv(
    "bases/cierre_patrimonio",
    i,
    ',',0).drop(columns=["Unnamed: 0.1"])
    for i in list_name_file_cierre_patrimonio
    ]
df_cierre_patrimonio = \
    pd.concat(df_list_cierre_patrimonio)
del df_list_cierre_patrimonio

Dimensiones: (30674, 4)
Columnas 
 Unnamed: 0.1, patrimonio, fecha_cierre, id
Dimensiones: (31111, 4)
Columnas 
 Unnamed: 0.1, patrimonio, fecha_cierre, id
Dimensiones: (31425, 4)
Columnas 
 Unnamed: 0.1, patrimonio, fecha_cierre, id
Dimensiones: (31997, 4)
Columnas 
 Unnamed: 0.1, patrimonio, fecha_cierre, id
Dimensiones: (32423, 4)
Columnas 
 Unnamed: 0.1, patrimonio, fecha_cierre, id
Dimensiones: (33010, 4)
Columnas 
 Unnamed: 0.1, patrimonio, fecha_cierre, id
Dimensiones: (33994, 4)
Columnas 
 Unnamed: 0.1, patrimonio, fecha_cierre, id
Dimensiones: (34762, 4)
Columnas 
 Unnamed: 0.1, patrimonio, fecha_cierre, id
Dimensiones: (35420, 4)
Columnas 
 Unnamed: 0.1, patrimonio, fecha_cierre, id
Dimensiones: (36341, 4)
Columnas 
 Unnamed: 0.1, patrimonio, fecha_cierre, id
Dimensiones: (37027, 4)
Columnas 
 Unnamed: 0.1, patrimonio, fecha_cierre, id
Dimensiones: (36444, 4)
Columnas 
 Unnamed: 0.1, patrimonio, fecha_cierre, id
Dimensiones: (35956, 4)
Columnas 
 Unnamed: 0.1, patrimonio, fec

## set columns format

In [12]:
exp_lib.columns_cast(
    aportes_rescates_12m,
    ["n_rescates","n_aportes"],
    "float64")
exp_lib.columns_cast(
    aportes_rescates_12m,
    ["id"],
    "int64")
aportes_rescates_12m.head(2)

n_rescates    float64
n_aportes     float64
id             object
dtype: object
n_rescates    float64
n_aportes     float64
id              int64
dtype: object


,n_rescates,n_aportes,id
0,0.0,0.0,0
1,7.0,11.0,1


In [13]:
exp_lib.columns_cast(
    cuenta_banco,
    ["id"],
    "int64")
cuenta_banco.head(2)

id                        int64
clasificacion_cliente    object
dtype: object


,id,clasificacion_cliente
0,37146,segmento_1
1,37147,segmento_1


In [14]:
demograficos["fec_nacim"]=demograficos["fec_nacim"].apply(exp_lib.convert_to_date)
print(
    f"""
    Formato fecha a periodo
    {''.join(str(demograficos["fec_nacim"].to_list()[0]).split('-')[:2])}
    """
    )
exp_lib.columns_cast(
    demograficos,
    ["id"],
    "int64")
demograficos.head(2)


    Formato fecha a periodo
    194304
    
id                                int64
fec_nacim                datetime64[ns]
flg_empleado                     object
gls_nacioncorta                  object
gls_residencialarga              object
gls_profesionlarga               object
cod_sexo                         object
gls_cod_estcivillarga            object
gls_niv_educaclarga              object
gls_sep_bieneslarga              object
dtype: object


,id,fec_nacim,flg_empleado,gls_nacioncorta,gls_residencialarga,gls_profesionlarga,cod_sexo,gls_cod_estcivillarga,gls_niv_educaclarga,gls_sep_bieneslarga
0,10170,1943-04-23,N,Chile,Residencia Propia,PERIODISTA,F,Casado,Universitario Completo,SIN SEPARACIÓN
1,24533,1957-01-18,N,Chile,,OTRA,F,Casado,Medio,SIN SEPARACIÓN


In [15]:
deudas.columns

Index(['id', 'periodo', 'entidades', 'disponible', 'deuda_1', 'deuda_2',
       'deuda_3', 'deuda_4', 'deuda_5', 'mora_1', 'mora_2', 'mora_3', 'mora_4',
       'mora_5', 'mora_6', 'mora_7'],
      dtype='object')

In [16]:
list_name_file_1
# deudas
exp_lib.columns_cast(
    deudas,
    ['disponible', 'deuda_1', 'deuda_2',
       'deuda_3', 'deuda_4', 'deuda_5', 'mora_1', 'mora_2', 'mora_3', 'mora_4',
       'mora_5', 'mora_6', 'mora_7',],
    "float64")
exp_lib.columns_cast(
    deudas,
    ['id', 'periodo','entidades'],
    "int64")

id             object
periodo        object
entidades      object
disponible    float64
deuda_1       float64
deuda_2       float64
deuda_3       float64
deuda_4       float64
deuda_5       float64
mora_1        float64
mora_2        float64
mora_3        float64
mora_4        float64
mora_5        float64
mora_6        float64
mora_7        float64
dtype: object
id              int64
periodo         int64
entidades       int64
disponible    float64
deuda_1       float64
deuda_2       float64
deuda_3       float64
deuda_4       float64
deuda_5       float64
mora_1        float64
mora_2        float64
mora_3        float64
mora_4        float64
mora_5        float64
mora_6        float64
mora_7        float64
dtype: object


In [17]:
# list_name_file_1
exp_lib.columns_cast(
    id_label,
    ['id', 'label'],
    "int64")

id       int64
label    int64
dtype: object


In [21]:
df_cierre_patrimonio.head(2)

,patrimonio,fecha_cierre,id
0,11.80456920962544,2022-02-28,21636
1,3.66808983990638,2022-02-28,707


In [23]:
df_cierre_patrimonio["fecha_cierre"]=\
    df_cierre_patrimonio["fecha_cierre"].apply(exp_lib.convert_to_date)
exp_lib.columns_cast(
    df_cierre_patrimonio,
    ["patrimonio"],
    "float64")
exp_lib.columns_cast(
    df_cierre_patrimonio,
    ["id"],
    "int64")
df_cierre_patrimonio.head(2)

patrimonio             float64
fecha_cierre    datetime64[ns]
id                      object
dtype: object
patrimonio             float64
fecha_cierre    datetime64[ns]
id                       int64
dtype: object


,patrimonio,fecha_cierre,id
0,11.804569,2022-02-28,21636
1,3.668090,2022-02-28,707


In [24]:
df_cierre_patrimonio.fecha_cierre.max()

Timestamp('2023-02-28 00:00:00')